# Explorando estatísticas de gênero do Banco Mundial

Nas aulas anteriores, acessamos fontes de dados criando, nós mesmos, funções ou rotinas de consulta a suas API. 

Algumas fontes, no entanto, oferecem bibliotecas para facilitar as definições de parâmetros e criação de consultas, integrando-as mais suavemente ao nosso código.

O Banco Mundial oferece biblioteca em Python para consultas aos seus bancos de dados:

https://pypi.org/project/wbgapi/

https://blogs.worldbank.org/opendata/introducing-wbgapi-new-python-package-accessing-world-bank-data

In [ ]:
# !pip install wbgapi

In [ ]:
import wbgapi as wb
import pandas as pd

Podemos listas as diversas fontes de dados disponíveis com a seguinte função:

In [ ]:
wb.source.info()

Vamos consultas as séries de dados do banco "Gender Statistics":

In [ ]:
wb.series.info(db=14)

As tabelas 'SL.TLF.ACTI.FE.ZS' e 'SL.TLF.ACTI.MA.ZS' referem-se à parcela da população feminina e masculina, respectivamente, de 15 a 64 anos que integra a força de trabalho 

A função de consulta 'wb.data.DataFrame()' retorna, por padrão, um Pandas DataFrame em que os índices são o código ISO3 de país, região ou grupo de renda, e as colunas são anos. 

Vamos consultar as séries indicadas restringindo o período a um só ano, e transformá-las em Pandas Series.

In [ ]:
female_labor_force = wb.data.DataFrame('SL.TLF.ACTI.FE.ZS', time = 2019).squeeze()
male_labor_force = wb.data.DataFrame('SL.TLF.ACTI.MA.ZS', time = 2019).squeeze()

In [ ]:
female_labor_force

Retorne a taxa de participação na força de trabalho feminina e masculina para o Brasil (código "BRA") e para mundo (código "WLD").

In [ ]:
income_info

In [ ]:
female_labor_force[income_levels]

In [ ]:
print(f"Participação feminina no Brasil: {female_labor_force['BRA']:.2f}%")
print(f"Participação masculina no Brasil: {male_labor_force['BRA']:.2f}%")
      
print(f"Participação feminina no mundo: {female_labor_force['WLD']:.2f}%")
print(f"Participação masculina no mundo: {male_labor_force['WLD']:.2f}%")

Como vimos, as linhas e índices da série incluem tanto países como agregações por região e grupo de renda. Antes de prosseguir com a análise, vamos separá-los.

In [ ]:
economy_info = wb.economy.info()
region_info = wb.region.info()
income_info = wb.income.info()

In [ ]:
country_codes = [pais.get('id') 
                 for pais in economy_info.items
                 if pais.get('region') != ''] # códigos que não são países têm o campo region e income em branco

income_levels = [item.get('id') for item in income_info.items]

In [ ]:
income_levels

In [ ]:
paises_info = [{'id': pais.get('id'),
                'name': pais.get('value'),
                'region': pais.get('region'),
                'income_level': pais.get('incomeLevel')}
                for pais in economy_info.items 
                if pais.get("region") != ""]

Com os objetos que já temos, crie a série female_labor_force_income e male_labor_force_income, subconjuntos das séries completas onde os índices são somente os códigos dos grupos de renda.

Em seguida, crie as séries female_labor_force_country e male_labor_force_country, somente com códigos de países.

In [ ]:
female_labor_force_income = female_labor_force[income_levels]
male_labor_force_income = male_labor_force[income_levels]

In [ ]:
female_labor_force_country = female_labor_force[country_codes]
male_labor_force_country = male_labor_force[country_codes]

Agora junte as séries em dois DataFrames: um com as participações feminina e masculina por grupo de renda, outro por país. Adicione também, em cada DataFrame, uma coluna com a diferença entre as taxas.

In [ ]:
labor_force_income = pd.DataFrame({"female_rate": female_labor_force_income,
                                   "male_rate": male_labor_force_income,
                                   "rate_gap": male_labor_force_income - female_labor_force_income})

In [ ]:
labor_force_income

In [ ]:
labor_force_income.describe()

In [ ]:
type(male_labor_force_country - female_labor_force_country)

In [ ]:
labor_force_country = pd.DataFrame({"female_rate": female_labor_force_country,
                                    "male_rate": male_labor_force_country,
                                    "rate_gap": male_labor_force_country - female_labor_force_country})

In [ ]:
labor_force_country

Encontre a média, mediana, mínimo e máximo das participações e da diferença, dentre os países.

In [ ]:
labor_force_country.mean()

In [ ]:
labor_force_country.median(axis = "index")

In [ ]:
labor_force_country.max(axis = "index")

In [ ]:
labor_force_country.min(axis = "index")

In [ ]:
labor_force_income['rate_gap'].min()

In [ ]:
labor_force_income.loc[labor_force_income['rate_gap'] == labor_force_income['rate_gap'].max()]

Encontre qual o grupo de renda com menor diferença entre as taxas de participação na força de trabalho.

In [ ]:
labor_force_income['rate_gap'].idxmin()

Encontre a média da taxa de participação feminina na força de trabalho entre os países Brasil, Argentina, Uruguai e Paraguai, e entre os países Canadá e Estados Unidos.

In [ ]:
[pais.get('id') 
 for pais in paises_info 
 if pais.get('name') in ['Brazil', 'Argentina', 'Uruguay', 'Paraguay']]

In [ ]:
female_labor_force_country[['BRA', 'ARG', 'URY', 'PRY']].mean()

In [ ]:
female_labor_force_country[['USA', 'CAN']].mean()